In [1]:
import sys
sys.path.append("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/Mycode/")

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats

#importing all usefull lib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, train_test_split, KFold, cross_val_predict
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
%matplotlib inline
from sklearn.linear_model import LinearRegression, RidgeCV, Lasso, ElasticNetCV, BayesianRidge, LassoLarsIC
from sklearn.metrics import mean_squared_error, make_scorer 
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import math
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats

In [3]:
train_dataSet=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/train.csv")
test_dataSet=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/test.csv")

In [4]:
train_dataSet=train_dataSet[train_dataSet.GrLivArea < 4500]
print(train_dataSet.shape)
train_dataSet.reset_index(drop=True, inplace=True)
Sale_Price=train_dataSet.iloc[:,80]
Sale_Price.shape
Sale_Pricedf=pd.DataFrame(Sale_Price)

(1458, 81)


In [5]:
#3.1 data cleaning: no correct answer
# how many % of data are missing
total = Sale_Pricedf.isnull().sum().sort_values(ascending=False)
percent = (Sale_Pricedf.isnull().sum()/Sale_Pricedf.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
# No missing checked

,Total,Percent
SalePrice,0,0.0


In [6]:
train_dataSet.shape

(1458, 81)

In [7]:
test_dataSet.shape

(1459, 80)

In [8]:
Sale_Price.shape

(1458,)

In [9]:
train=train_dataSet.drop(["SalePrice"],axis=1)# combine test & train
train_data=pd.concat([train,test_dataSet],keys=['train_part','test_part'])
# train_data=pd.concat({'x':train,'y':test_dataSet})
X=train_data.loc["train_part"] # 这样就是runnable了
print(X.shape)
# train_data.index=train_data["Id"]
# train_data=train_data.drop(["Id"],axis=1)

(1458, 80)


In [10]:
train_data.shape

(2917, 80)

In [11]:
#3.1 data cleaning: no correct answer
# how many % of data are missing
total = X.isnull().sum().sort_values(ascending=False)
percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
# No missing checked

,Total,Percent
PoolQC,1452,0.995885
MiscFeature,1404,0.962963
Alley,1367,0.937586
Fence,1177,0.807270
FireplaceQu,690,0.473251
LotFrontage,259,0.177641
GarageCond,81,0.055556
GarageType,81,0.055556
GarageYrBlt,81,0.055556
GarageFinish,81,0.055556


In [12]:
X=train_data.loc["train_part"] # 这样就是runnable了
print(X.shape) #是这样吗！！！！！

(1458, 80)


In [13]:
train_data=train_data.drop(["Id"],axis=1)# combine test & train


In [14]:
# numerical -> categorical
train_data = train_data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45",50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},"MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}})                        

In [15]:
train_data['YrSold'] = train_data['YrSold'].astype(str)# numerical -> categorical
train_data['MoSold'] = train_data['MoSold'].astype(str)# numerical -> categorical

# Missing Data Handling Begin

In [16]:
train_data['Functional'] = train_data['Functional'].fillna('Typ') #Home functionality (Assume typical unless deductions are warranted)
train_data['Exterior1st'] = train_data['Exterior1st'].fillna(train_data['Exterior1st'].mode()[0]) #Exterior covering on house
train_data['Exterior2nd'] = train_data['Exterior2nd'].fillna(train_data['Exterior2nd'].mode()[0])
# Exterior covering on house
train_data['SaleType'] = train_data['SaleType'].fillna(train_data['SaleType'].mode()[0])

In [17]:
#GarageYrBlt是因为没有车库还是缺失？
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    train_data[col] = train_data[col].fillna(0)
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    train_data[col] = train_data[col].fillna('None')
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    train_data[col] = train_data[col].fillna('None')

In [18]:
# MSSubClass: Identifies the type of dwelling involved in the sale.	
# 将MSZoning和MSSubClass相关联
train_data['MSZoning'] = train_data.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))

In [19]:
#3.1 data cleaning: no correct answer
# how many % of data are missing
total = train_data.isnull().sum().sort_values(ascending=False)
percent = (train_data.isnull().sum()/train_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
PoolQC,2908,0.996915
MiscFeature,2812,0.964004
Alley,2719,0.932122
Fence,2346,0.804251
FireplaceQu,1420,0.486802
LotFrontage,486,0.166610
MasVnrType,24,0.008228
MasVnrArea,23,0.007885
BsmtFullBath,2,0.000686
Utilities,2,0.000686


In [20]:
objects = []
for i in train_data.columns:
    if train_data[i].dtype == object:
        objects.append(i)
train_data.update(train_data[objects].fillna('None'))
#LotFrontage: Linear feet of street connected to property
train_data['LotFrontage'] = train_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = []
for i in train_data.columns:
    if train_data[i].dtype in numeric_dtypes:
        numerics.append(i)
train_data.update(train_data[numerics].fillna(0))

In [21]:
#3.1 data cleaning: no correct answer
# how many % of data are missing
total = train_data.isnull().sum().sort_values(ascending=False)
percent = (train_data.isnull().sum()/train_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
# No missing checked

,Total,Percent
SaleCondition,0,0.0
Foundation,0,0.0
RoofMatl,0,0.0
Exterior1st,0,0.0
Exterior2nd,0,0.0
MasVnrType,0,0.0
MasVnrArea,0,0.0
ExterQual,0,0.0
ExterCond,0,0.0
BsmtQual,0,0.0


In [22]:
train_data.shape # 缺失值填补是没有行数变换的

(2917, 79)

# Missing Data Handling End
# Skewness Data Handling Begin

In [23]:
num_col=train_data._get_numeric_data().columns.tolist()
cat_col=set(train_data.columns)-set(num_col)
print(len(num_col)+len(cat_col)) #未dummy前时刻check和是79

79


In [24]:
train_cat = train_data[cat_col]
pct=[]
for ix in train_cat.columns:
    temp=train_cat[ix].describe()
    # print(temp.describe)
    # break
    pct.append(temp["freq"]/temp["count"])
skewData=pd.DataFrame(pct,index=train_cat.columns,columns=["skewness"])
skewData=skewData.sort_values(by="skewness",ascending=False)
# print (skewData)

In [25]:
train_data = train_data.drop((skewData[skewData['skewness'] >= 0.95]).index,1) # 行不变 只是列变了 drop掉了一些features。

In [26]:
train_data.shape # 这里因为skewness行变了我怎么再找到test呢？

(2917, 71)

In [27]:
#2.2: some categorical features when there is information in the order
#Alley: Type of alley access to property
train_data=train_data.replace({"Alley":{"Grvl" : 1, "Pave" : 2}})
#BsmtCond: Evaluates the general condition of the basement
train_data=train_data.replace({"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#BsmtExposure: Refers to walkout or garden level walls
train_data=train_data.replace({"BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3}})
#BsmtFinType1: Rating of basement finished area
train_data=train_data.replace({"BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6}})
#BsmtFinType2: Rating of basement finished area (if multiple types)
train_data=train_data.replace({"BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6}})
#BsmtQual: Evaluates the height of the basement
train_data=train_data.replace({"BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})
#ExterCond: Evaluates the present condition of the material on the exterior
train_data=train_data.replace({"ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}})
#ExterQual: Evaluates the quality of the material on the exterior 
train_data=train_data.replace({"ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}})
#FireplaceQu: Fireplace quality
train_data=train_data.replace({"FireplaceQu" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#Functional: Home functionality (Assume typical unless deductions are warranted)
train_data=train_data.replace({"Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5,"Min2" : 6, "Min1" : 7, "Typ" : 8}})
#HeatingQC: Heating quality and condition
train_data=train_data.replace({"HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#KitchenQual: Kitchen quality
train_data=train_data.replace({"KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#LandSlope: Slope of property
train_data=train_data.replace({"LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3}})
#LotShape: General shape of property
train_data=train_data.replace({"LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4}})
#PavedDrive: Paved driveway
train_data=train_data.replace({"PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2}})

#2.3: Simplifications of existing categorical 
train_data["Condition1"] = train_data.Condition1.replace({"RRNe" : "Other",  "RRNn" : "Other","PosA" : "Other", "RRAe" : "Other"
})
train_data["Electrical"] = train_data.Electrical.replace({"Mix" : "Other", "FuseP" : "Other"})
train_data["Exterior1st"] = train_data.Exterior1st.replace({"AsphShn" : "Other","CBlock" : "Other","ImStucc" : "Other",  "BrkComm" : "Other","Stone" : "Other"})
train_data["Exterior2nd"] = train_data.Exterior2nd.replace({"CBlock" : "Other", "AsphShn" : "Other","Stone" : "Other", "Brk Cmn" : "Other","ImStucc" : "Other"})
train_data["Foundation"] = train_data.Foundation.replace({"Wood" : "Other", "Stone" : "Other" })
train_data["HouseStyle"] = train_data.HouseStyle.replace({"2.5Fin" : "Other", "2.5Unf" : "Other", "1.5Unf" : "Other"})
train_data["LotConfig"] = train_data.LotConfig.replace({"FR3" : "FR2" })
train_data["MSSubClass"] = train_data.MSSubClass.replace({"SC40" : "Other",  "SC180" : "Other", "SC45" : "Other", "SC75" : "Other"})
train_data["MSZoning"] = train_data.MSZoning.replace({"C (all)" : "Other",                               "RH" : "Other"})
train_data["Neighborhood"] = train_data.Neighborhood.replace({"Blueste" : "Other","NPkVill" : "Other", "Veenker" : "Other"  })
train_data["RoofStyle"] = train_data.RoofStyle.replace({"Shed" : "Other",   "Mansard" : "Other","Gambrel" : "Other","Flat" : "Other"})
train_data["SaleCondition"] = train_data.SaleCondition.replace({"AdjLand" : "Other", "Alloca" : "Other"})
train_data["SaleType"] = train_data.SaleType.replace({"Con" : "Other", "Oth" : "Other","CWD" : "Other", "ConLI" : "Other",  "ConLw" : "Other","ConLD" : "Other"  })

In [28]:
#2.4 Combinations of existing features
# Overall quality of the house
train_data["OverallGrade"] = train_data["OverallQual"] * train_data["OverallCond"]
# Overall quality of the exterior
train_data["ExterGrade"] = train_data["ExterQual"] * train_data["ExterCond"]
# Overall kitchen score
train_data["KitchenScore"] = train_data["KitchenAbvGr"] * train_data["KitchenQual"]
# Total number of bathrooms
train_data["TotalBath"] = train_data["BsmtFullBath"] + (0.5 * train_data["BsmtHalfBath"]) + \
train_data["FullBath"] + (0.5 * train_data["HalfBath"])
train_data.drop(["BsmtFullBath","BsmtHalfBath","FullBath","HalfBath"],axis=1)
# Total SF for house (incl. basement)
train_data["AllSF"] = train_data["GrLivArea"] + train_data["TotalBsmtSF"]
train_data.drop(["GrLivArea","TotalBsmtSF"],axis=1)
# Total SF for 1st + 2nd floors
train_data["AllFlrsSF"] = train_data["1stFlrSF"] + train_data["2ndFlrSF"]
train_data.drop(["1stFlrSF","2ndFlrSF"],axis=1)
# Total SF for porch
train_data["AllPorchSF"] = train_data["OpenPorchSF"] + train_data["EnclosedPorch"] + \
train_data["3SsnPorch"] + train_data["ScreenPorch"]
train_data.drop(["OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"],axis=1)

MSSubClass MSZoning  LotFrontage  LotArea Alley  LotShape  \
train_part 0          SC60       RL         65.0     8450  None         4   
           1          SC20       RL         80.0     9600  None         4   
           2          SC60       RL         68.0    11250  None         3   
           3          SC70       RL         60.0     9550  None         3   
           4          SC60       RL         84.0    14260  None         3   
...                    ...      ...          ...      ...   ...       ...   
test_part  1454      SC160       RM         21.0     1936  None         4   
           1455      SC160       RM         21.0     1894  None         4   
           1456       SC20       RL        160.0    20000  None         4   
           1457       SC85       RL         62.0    10441  None         4   
           1458       SC60       RL         74.0     9627  None         4   

                LandContour LotConfig Neighborhood Condition1  ... YrSold  \
train_part 0            Lvl    Inside      CollgCr       Norm  ...   2008   
           1            Lvl       FR2        Other      Feedr  ...   2007   
           2            Lvl    Inside      CollgCr       Norm  ...   2008   
           3            Lvl    Corner      Crawfor       Norm  ...   2006   
           4            Lvl       FR2      NoRidge       Norm  ...   2008   
...                     ...       ...          ...        ...  ...    ...   
test_part  1454         Lvl    Inside      MeadowV       Norm  ...   2006   
           1455         Lvl    Inside      MeadowV       Norm  ...   2006   
           1456         Lvl    Inside      Mitchel       Norm  ...   2006   
           1457         Lvl    Inside      Mitchel       Norm  ...   2006   
           1458         Lvl    Inside      Mitchel       Norm  ...   2006   

                SaleType  SaleCondition  OverallGrade  ExterGrade  \
train_part 0          WD         Normal            35          12   
           1          WD         Normal            48           9   
           2          WD         Normal            35          12   
           3          WD        Abnorml            35           9   
           4          WD         Normal            40          12   
...                  ...            ...           ...         ...   
test_part  1454       WD         Normal            28           9   
           1455       WD        Abnorml            20           9   
           1456       WD        Abnorml            35           9   
           1457       WD         Normal            25           9   
           1458       WD         Normal            35           9   

                 KitchenScore TotalBath   AllSF AllFlrsSF AllPorchSF  
train_part 0                4       3.5  2566.0      1710         61  
           1                3       2.5  2524.0      1262          0  
           2                4       3.5  2706.0      1786         42  
           3                4       2.0  2473.0      1717        307  
           4                4       3.5  3343.0      2198         84  
...                       ...       ...     ...       ...        ...  
test_part  1454             3       1.5  1638.0      1092          0  
           1455             3       1.5  1638.0      1092         24  
           1456             3       2.0  2448.0      1224          0  
           1457             3       1.5  1882.0       970         32  
           1458             3       2.5  2996.0      2000         48  

[2917 rows x 74 columns]

In [29]:
train_data["OverallQual-s2"] = train_data["OverallQual"] ** 2
train_data["AllSF-2"] = train_data["AllSF"] ** 2
train_data["AllFlrsSF-2"] = train_data["AllFlrsSF"] ** 2
train_data["GrLivArea-2"] = train_data["GrLivArea"] ** 2

In [30]:
# Differentiate numerical features and categorical features
categorical_features = train_data.select_dtypes(include = ["object"]).columns
numerical_features = train_data.select_dtypes(exclude = ["object"]).columns
print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
train_num = train_data[numerical_features]
train_cat = train_data[categorical_features]
print("NAs for numerical features in train : " + str(train_num.isnull().values.sum()))
train_num = train_num.fillna(train_num.median())
print("Remaining NAs for numerical features in train : " + str(train_num.isnull().values.sum()))

Numerical features : 49
Categorical features : 33
NAs for numerical features in train : 0
Remaining NAs for numerical features in train : 0


In [31]:
train_data.shape

(2917, 82)

In [32]:
#2.6 turn category features to dummy
print("NAs for categorical features in train : " + str(train_cat.isnull().values.sum()))
train_cat = pd.get_dummies(train_cat, drop_first=True)
print("Remaining NAs for categorical features in train : " + str(train_cat.isnull().values.sum()))

NAs for categorical features in train : 0
Remaining NAs for categorical features in train : 0


In [33]:
#2.7 join categorical and numerical features 
train_Data_New = pd.concat([train_num, train_cat], axis = 1)
print("New number of features : " + str(train_Data_New.shape[1]))

New number of features : 228


In [34]:
train_Data_New.shape

(2917, 228)

In [35]:
#2.8 remove collinear columns
# Create correlation matrix
corr_matrix = train_Data_New.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
train_Data_New.drop(to_drop, axis=1, inplace=True)

In [36]:
train_Data_New.shape

(2917, 205)

In [37]:
# 日哦 使用loc函数，索引的是字符串，前后都要取，是属于“前闭后闭”的情况
# iloc函数索引的数据是int整型，因此是Python默认的前闭后开
# Python默认的前闭后开

In [38]:
# Train_data=train_Data_New.iloc[:1458]#这里这个index的索引好奇怪 取前1458行
# Test_data=train_Data_New.iloc[1458:]

In [39]:
Train_data=train_Data_New.loc["train_part"] 
Test_data=train_Data_New.loc["test_part"]

In [40]:
Train_data.shape

(1458, 205)

In [41]:
Train_data.shape

(1458, 205)

In [42]:
Test_data.shape

(1459, 205)

In [43]:
Sale_Price.shape

(1458,)

In [44]:
Train_data.insert(0,column="SalePrice",value=Sale_Price)
Train_data.shape

(1458, 206)

In [45]:
dependentV=Train_data["SalePrice"]
Train_data.drop(["SalePrice"],axis=1, inplace=True)# 
Train_data.shape

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(1458, 205)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(Train_data, dependentV, test_size = 0.3, random_state = 0)
print("X_train : " + str(X_train.shape))
print("X_test : " + str(X_test.shape))
print("y_train : " + str(y_train.shape))
print("y_test : " + str(y_test.shape))

X_train : (1020, 205)
X_test : (438, 205)
y_train : (1020,)
y_test : (438,)


In [47]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from Help_functions_v2 import sklearn_Pvalue, sklearn_adjR2, RMSE
import numpy as np

In [48]:
#3.1 build multi linear regression 
lr=LinearRegression()
lr.fit(X_train, y_train)
y_trainPred=lr.predict(X_train)
#look at r_squared, adjusted r_squared
print("r-squared of in-sample is")
r2_score(y_train, y_trainPred)
adj_r2=sklearn_adjR2(X_train, y_train)
print("adjusted r-squared of in-sample is")
print(adj_r2)
print("RMSE of in-sample is")
print(RMSE(y_trainPred,y_train))

r-squared of in-sample is
adjusted r-squared of in-sample is
0.925100087902679
RMSE of in-sample is
19098.022413377636


In [49]:
# 3.2 out of sample
y_testPred=lr.predict(X_test)
#look at  r_squared, adjusted r_squared
print("r-squared of out-of-sample is")
r2_score(y_test, y_testPred)
print("adjusted r-squared of out-of-sample is")
adj_r2=sklearn_adjR2(X_test, y_test)
print("RMSE of out-of-sample is")
print(RMSE(y_testPred, y_test))


r-squared of out-of-sample is
adjusted r-squared of out-of-sample is
RMSE of out-of-sample is
25818.99667779577


In [50]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, SelectKBest
import numpy as np
from Help_functions_v2 import sklearn_Pvalue, sklearn_adjR2, RMSE

In [51]:
#3 feature selection with regularization
#3.1 Ridge Regularization
ridge_model=Ridge(alpha=1)
ridge_model.fit(X_train, y_train)
y_trainPred=ridge_model.predict(X_train)
#in sample
print("in-sample r-squared is")
print(r2_score(y_train, y_trainPred))
print ("RMSE is of in-sample")
print(RMSE(y_trainPred,y_train))
#out of sample
y_testPred=ridge_model.predict(X_test)
print("out-of-sample r-squared is")
print(r2_score(y_test, y_testPred))
print ("RMSE is out-of-sample")
print(RMSE(y_testPred,y_test))
print(ridge_model.score(X_test,y_test))

in-sample r-squared is
0.938715299195137
RMSE is of in-sample
19340.40654756993
out-of-sample r-squared is
0.9040136188447268
RMSE is out-of-sample
25562.152443381507
0.9040136188447268


In [52]:
#3.2 Lasso Regularization
lasso_model=Lasso(alpha=1)
lasso_model.fit(X_train, y_train)
y_trainPred=lasso_model.predict(X_train)

#in sample
print("lasso in-sample r-squared is")
print(r2_score(y_train, y_trainPred))
print ("Lasso RMSE of in-sample is ")
print(RMSE(y_trainPred,y_train))
#out of sample
y_testPred=lasso_model.predict(X_test)
print("Lasso out-of-sample r-squared is")
print(r2_score(y_test, y_testPred))
print ("Lasso RMSE out-of-sample is")
print(RMSE(y_testPred,y_test))
print(lasso_model.score(X_test,y_test)) #沃日？
#lasso could not converge

lasso in-sample r-squared is
0.9401940547011193
Lasso RMSE of in-sample is 
19105.6467432301
Lasso out-of-sample r-squared is
0.9024946529304917
Lasso RMSE out-of-sample is
25763.61659792456
0.9024946529304917
/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 187554299825.78192, tolerance: 622557256.4164598
  model = cd_fast.enet_coordinate_descent(


In [53]:
rfr=RandomForestRegressor(n_estimators = 50,random_state=40,min_impurity_decrease=0.002,min_weight_fraction_leaf=0.001,min_samples_split=5)
rfr.fit(X_train, y_train)
y_predictrfc = rfr.predict(X_test)
#here we can check our model score
print(rfr.score(X_test,y_test)) #沃日？

0.9167099786564366


In [54]:
rmse = math.sqrt(mean_squared_error(y_test, rfr.predict(X_test)))
print("mear squares error :",rmse)

mear squares error : 23811.628487000555


In [55]:
GBoost= GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt',min_samples_leaf=15, min_samples_split=10,loss='huber', random_state =42)
                                   #RMSE estimated through the partition of the train set
GBoost.fit(X_train, y_train)
# y_predictrfc = rfr.predict(X_test)
#here we can check our model score
print(GBoost.score(X_test,y_test)) #沃日？ GBoost反倒没有randomForest高？

0.9235231179825951


In [56]:
xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460, max_depth=3, min_child_weight=0,
gamma=0, subsample=0.7,colsample_bytree=0.7,objective='reg:linear', nthread=-1,scale_pos_weight=1, seed=27, reg_alpha=0.00006)
xgboost.fit(X_train,y_train) 
print(xgboost.score(X_test,y_test))                            

[12:15:16] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
0.9344686575631225
/usr/local/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [57]:
lightgbm = LGBMRegressor(objective='regression',num_leaves=4, learning_rate=0.01, 
n_estimators=5000, max_bin=200, bagging_fraction=0.75,bagging_freq=5,  bagging_seed=7, feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )
lightgbm.fit(X_train,y_train)
print(lightgbm.score(X_test,y_test))

[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
0.9227558791821846


In [58]:
# y_model_prerfc=ridge_model.predict(Test_data)
# y_model_prerfc=np.around(y_model_prerfc,0)
# prediction=np.array(y_model_prerfc).tolist()
# 0.15 

In [59]:
# rfr
# y_model_prerfc=rfr.predict(Test_data)
# y_model_prerfc=np.around(y_model_prerfc,0)
# prediction=np.array(y_model_prerfc).tolist()
# 0.14

In [60]:
# y_model_prerfc=lr.predict(Test_data)
# y_model_prerfc=np.around(y_model_prerfc,0)
# prediction=np.array(y_model_prerfc).tolist()
# 0.16

In [61]:
# y_model_prerfc=GBoost.predict(Test_data)
# y_model_prerfc=np.around(y_model_prerfc,0)
# prediction=np.array(y_model_prerfc).tolist()
# 0.13

In [62]:
# xgboost
y_model_prerfc=xgboost.predict(Test_data)
y_model_prerfc=np.around(y_model_prerfc,0)
prediction=np.array(y_model_prerfc).tolist()

/usr/local/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [63]:
test_dataSet.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [64]:
print("before inserting: ",test_dataSet.shape)
test_dataSet.insert(1,column="SalePrice",value=prediction)
print("After inserting: ",test_dataSet.shape)

before inserting:  (1459, 80)
After inserting:  (1459, 81)


In [65]:
predict_sub=test_dataSet.drop(test_dataSet.iloc[:,2:],axis=1)
predict_sub.head()

,Id,SalePrice
0,1461,120583.0
1,1462,161356.0
2,1463,188057.0
3,1464,191204.0
4,1465,194520.0


In [67]:
predict_sub.to_csv('Home_predictionsXGboost.csv',index=False)
